In [1]:
import numpy as np
import pyfits
import math
from math import log10
import matplotlib.pyplot as plt
import scipy
from scipy.stats import binned_statistic
from scipy.stats import binned_statistic_2d
from matplotlib import rc
import pickle
import os
import astropy as ap
from astropy.cosmology import WMAP7
from scipy import interpolate
import corner
import matplotlib.patheffects as patheffects
from matplotlib import rcParams
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib as mpl
from matplotlib import rc
from matplotlib import colors
from __future__ import unicode_literals
# rcParams["font.size"] = 16
rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Computer Modern Sans"]
rcParams["text.usetex"] = True
rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"

rcParams['xtick.major.pad']='10'
rcParams['ytick.major.pad']='10'

# rcParams['font.family'] = 'serif'
# rcParams['font.serif'] = 'Ubuntu'
# rcParams['font.monospace'] = 'Ubuntu Mono'
rcParams['font.size'] = 20
rcParams['axes.labelsize'] = 17
rcParams['axes.labelweight'] = 'bold'
rcParams['axes.titlesize'] = 16
rcParams['xtick.labelsize'] = 13
rcParams['ytick.labelsize'] = 13
rcParams['legend.fontsize'] = 15
rcParams['figure.titlesize'] = 1




In [5]:
os.chdir("/Users/nitya/Work/Paper2/mpl7/")

a = pyfits.open("drpall-v2_4_3.fits")
a = a[1].data
print(a.columns)



ColDefs(
    name = 'plate'; format = 'K'
    name = 'ifudsgn'; format = '21A'
    name = 'plateifu'; format = '11A'
    name = 'mangaid'; format = '11A'
    name = 'versdrp2'; format = '6A'
    name = 'versdrp3'; format = '6A'
    name = 'verscore'; format = '6A'
    name = 'versutil'; format = '7A'
    name = 'versprim'; format = '4A'
    name = 'platetyp'; format = '14A'
    name = 'srvymode'; format = '12A'
    name = 'objra'; format = 'D'
    name = 'objdec'; format = 'D'
    name = 'ifuglon'; format = 'D'
    name = 'ifuglat'; format = 'D'
    name = 'ifura'; format = 'D'
    name = 'ifudec'; format = 'D'
    name = 'ebvgal'; format = 'D'
    name = 'nexp'; format = 'K'
    name = 'exptime'; format = 'D'
    name = 'drp3qual'; format = 'K'
    name = 'bluesn2'; format = 'D'
    name = 'redsn2'; format = 'D'
    name = 'harname'; format = '35A'
    name = 'frlplug'; format = 'K'
    name = 'cartid'; format = '11A'
    name = 'designid'; format = 'K'
    name = 'cenra'; format = 'D

In [6]:
b = pyfits.open("platelist-v2_4_3.fits")
b = b[1].data
print(b.columns)

ColDefs(
    name = 'plate'; format = 'K'
    name = 'mjd'; format = 'K'
    name = 'apocomp'; format = '4A'
    name = 'status2d'; format = '64A'
    name = 'status3d'; format = '64A'
    name = 'drp2qual'; format = 'K'
    name = 'type'; format = '9A'
    name = 'complete'; format = '3A'
    name = 'nscigood'; format = 'K'
    name = 'nscibad'; format = 'K'
    name = 'versdrp2'; format = '6A'
    name = 'verscore'; format = '6A'
    name = 'versutil'; format = '7A'
    name = 'versprim'; format = '4A'
    name = 'cartid'; format = 'K'
    name = 'mapname'; format = '14A'
    name = 'survey'; format = '13A'
    name = 'platetyp'; format = '14A'
    name = 'srvymode'; format = '12A'
    name = 'radeg'; format = 'D'
    name = 'decdeg'; format = 'D'
    name = 'nexp'; format = 'K'
    name = 'totalexptime'; format = 'D'
    name = 'taibeg'; format = 'K'
    name = 'taiend'; format = 'D'
    name = 'airmass'; format = 'D'
    name = 'seeing'; format = 'D'
    name = 'transpar'; format =

In [17]:
id1 = a['nsa_nsaid_v1b']
print(len(id1))
print(len(id1[id1>0]))
print(id1[id1>0])

id1 = a['nsa_nsaid']
b = a['nsa_z']
print(len(id1))
print(len(id1[id1>0]))
print(len(b[id1<0]))
print(len(b[id1==0]))

id1 = a['nsa_version']
print(len(id1))
print(id1)

11465
34
[228137 116913 102256 150984 230855 151023 150990 102317 151006 102206
 102198 130514  63894 150980 230927 150795 102252 228195 130525 209153
 102195 150970 150782 102312 209134 228132 102246 150971 150892 102259
 102241 150982 102261 150981]
11465
4682
6719
64
11465
['v1_0_1' '-9999' 'v1_0_1' ... 'v1_0_1' 'v1_0_1' '-9999']


In [8]:
print(a['plate'][0:10], a['ifudsgn'][0:10], a['plateifu'][0:10])

[8935 9118 7977 9494 8705 9680 8150 9487 8335 8554] ['12701' '12701' '12701' '12701' '12701' '12701' '12701' '12701' '12701'
 '12701'] ['8935-12701' '9118-12701' '7977-12701' '9494-12701' '8705-12701'
 '9680-12701' '8150-12701' '9487-12701' '8335-12701' '8554-12701']


In [23]:
print(len(np.unique(a['nsa_nsaid'])))
print(len(np.unique(a['plateifu'])))

4609
11465


In [16]:
print(a[0])
print(a[1])

(8935, '12701', '8935-12701', '1-404249', 'v2_4_3', 'v2_4_3', 'v1_6_2', 'v5_5_32', 'v2_5', 'APOGEE-2&MaNGA', 'MaNGA dither', 194.523423939, 29.0173526307, 85.4656882782, 87.6076553974, 194.52342, 29.017353, 0.00884234, 9, 8100.82, 0, 26.412, 54.6103, 'ma087', 22, '4', 9555, 195.773, 28.079836, 1.00379, 1.0207, 1.05363, 1.4388, 1.6464, 1.77775, 0.778568, 0.894813, 0.908259, 57809, 57809, 57813, 2.77918, 2.69495, 2.66897, 2.65623, 4288, 0, 0, 1, 'plateTargets-1.par', 6460, 'J125805.61+290102.5', 127, 127, 0, 0.0256766, 0.0154936, 0.0240863, 0.0334392, 0.0359381, 0.0146675, 0.0285023, 0.67973, -999.0, -999.0, -999.0, -999.0, -999.0, 0.969534, 0.712336, 0.736879, 17, 4646, 2, 'v1_0_1', 427604, -999, 0.0256766, 0.0255916, array([-11.0607, -14.0814, -16.4717, -17.7214, -18.3099, -18.5468,
       -18.7128]), array([-10.5316, -13.9755, -16.4276, -17.6815, -18.2274, -18.3603,
       -18.6375]), array([4.32649e-01, 3.08539e+02, 5.89027e+01, 2.06363e+03, 1.92868e+03,
       1.97501e+03, 4.55756e+

In [18]:
print(a.columns)
print(a['plate'], a['ifudsgn'], a['mangaid'], a['srvymode'])

ColDefs(
    name = 'plate'; format = 'K'
    name = 'ifudsgn'; format = '21A'
    name = 'plateifu'; format = '11A'
    name = 'mangaid'; format = '11A'
    name = 'versdrp2'; format = '6A'
    name = 'versdrp3'; format = '6A'
    name = 'verscore'; format = '6A'
    name = 'versutil'; format = '7A'
    name = 'versprim'; format = '4A'
    name = 'platetyp'; format = '14A'
    name = 'srvymode'; format = '12A'
    name = 'objra'; format = 'D'
    name = 'objdec'; format = 'D'
    name = 'ifuglon'; format = 'D'
    name = 'ifuglat'; format = 'D'
    name = 'ifura'; format = 'D'
    name = 'ifudec'; format = 'D'
    name = 'ebvgal'; format = 'D'
    name = 'nexp'; format = 'K'
    name = 'exptime'; format = 'D'
    name = 'drp3qual'; format = 'K'
    name = 'bluesn2'; format = 'D'
    name = 'redsn2'; format = 'D'
    name = 'harname'; format = '35A'
    name = 'frlplug'; format = 'K'
    name = 'cartid'; format = '11A'
    name = 'designid'; format = 'K'
    name = 'cenra'; format = 'D

In [19]:
new_a = a[a['srvymode']== 'MaNGA dither']

In [20]:
print(new_a['srvymode'])

['MaNGA dither' 'MaNGA dither' 'MaNGA dither' ... 'MaNGA dither'
 'MaNGA dither' 'MaNGA dither']
